### Importando bibliotecas

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import requests, json # API Loteria Caixa Brasil
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
import tensorflow as tf
import scipy.stats as stats

### token API para extrair os dados

In [9]:
# Criando uma lista vazia
lista = []
# APIWEB da loteria. (Gratuita por um período no dia do teste)
token = 'SjcnDsTOl31wggw'

### Extraindo histórico

In [10]:
# Buscando os jogos através dos seu concurso
nomedaloteria = 'lotofacil'
lastconcurso = 2899 # Ultimo jogo (sorteio recente)
firstconcurso = 2492 # Historico mínimo (diferença de n°s de concursos, o exemplo pegou perto do máximo na API)
while lastconcurso >= firstconcurso:
    response = requests.get(f'https://apiloterias.com.br/app/resultado?loteria={nomedaloteria}&token={token}&concurso={lastconcurso}')
    # Adicionando dados a lista
    json_data = json.loads(response.content)
    dezenas = json_data['dezenas']
    # for dezena in dezenas:
    lista.append(dezenas)
    arr = np.array(lista)
    lastconcurso -= 1
print(np.array(arr))

[['02' '03' '05' ... '19' '22' '25']
 ['01' '06' '09' ... '22' '23' '24']
 ['01' '07' '09' ... '23' '24' '25']
 ...
 ['01' '02' '04' ... '23' '24' '25']
 ['02' '03' '06' ... '20' '21' '25']
 ['04' '07' '08' ... '22' '23' '24']]


### Visualizando

In [11]:
df = pd.DataFrame(np.array(arr))
df.head()
print(df)

     0   1   2   3   4   5   6   7   8   9   10  11  12  13  14
0    02  03  05  07  08  10  13  14  15  16  17  18  19  22  25
1    01  06  09  10  12  14  15  16  17  18  20  21  22  23  24
2    01  07  09  10  11  12  13  14  17  18  19  21  23  24  25
3    01  02  05  07  08  09  11  13  15  16  17  18  20  23  24
4    02  03  05  08  09  11  14  15  16  17  18  21  22  24  25
..   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
403  02  03  04  08  09  10  11  13  17  18  20  22  23  24  25
404  03  05  09  10  12  13  14  15  16  17  18  19  21  24  25
405  01  02  04  05  06  08  09  10  12  13  16  18  23  24  25
406  02  03  06  07  09  10  11  12  14  15  16  18  20  21  25
407  04  07  08  09  10  11  12  13  14  15  17  20  22  23  24

[408 rows x 15 columns]


### Modelando

In [12]:
# cria um objeto scaler para normalizar os dados do dataframe df
scaler = StandardScaler().fit(df.values)
# transforma os valores do dataframe df em valores normalizados e os armazena em um array numpy
transformed_dataset = scaler.transform(df.values)
# cria um novo dataframe com os valores normalizados e o mesmo índice do original df
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)
# obtém o número de linhas, colunas e o comprimento da janela dos dados normalizados do dataframe

number_of_rows = df.values.shape[0]
window_length = 10
number_of_features = df.values.shape[1]

# cria dois arrays vazios para armazenar os dados de treino e de teste do modelo
train = np.empty([number_of_rows-window_length, window_length, number_of_features], dtype=float)
label = np.empty([number_of_rows-window_length, number_of_features], dtype=float)
window_length = 10

# percorre as linhas do dataframe transformado e armazena os dados de treino e de teste nos arrays criados anteriormente
for i in range(0, number_of_rows-window_length):
    train[i] = transformed_df.iloc[i:i+window_length, 0: number_of_features]
    label[i] = transformed_df.iloc[i+window_length: i+window_length+1, 0: number_of_features]


### Visualizando o formato dos dados de treino e de teste

In [14]:
train.shape

(398, 10, 15)

In [15]:
label.shape

(398, 15)

### *Treinando modelo*

In [23]:
# define o tamanho do lote para o treinamento do modelo como 100
batch_size = 100

# inicializa o modelo sequencial do Keras para a rede neural recorrente bidirecional
modelo = Sequential()

# adiciona uma camada bidirecional de LSTM com 240 unidades ocultas, recebendo como entrada uma sequência de 10 sorteios com 5 características cada, e retornando a saída para a próxima camada LSTM bidirecional
modelo.add(Bidirectional(LSTM(240,
            input_shape=(window_length, number_of_features),
            return_sequences=True)))

# adiciona uma camada de dropout com 20% de probabilidade de desligar aleatoriamente as unidades da camada anterior, para evitar o sobreajuste do modelo 
modelo.add(Dropout(0.2))

# adiciona outra camada bidirecional de LSTM com 240 unidades ocultas, recebendo como entrada a saída da camada anterior, e retornando a saída para a próxima camada LSTM bidirecional
modelo.add(Bidirectional(LSTM(240,
            input_shape=(window_length, number_of_features),
            return_sequences=True)))
# adiciona uma camada de dropout com 20% de probabilidade de desligar aleatoriamente as unidades da camada anterior, para evitar o sobreajuste do modelo 
modelo.add(Dropout(0.2))

# adiciona outra camada bidirecional de LSTM com 240 unidades ocultas, recebendo como entrada a saída da camada anterior, e retornando a saída para a próxima camada LSTM bidirecional
modelo.add(Bidirectional(LSTM(240,
            input_shape=(window_length, number_of_features),
            return_sequences=True)))

# adiciona uma camada de dropout com 20% de probabilidade de desligar aleatoriamente as unidades da camada anterior, para evitar o sobreajuste do modelo
modelo.add(Bidirectional(LSTM(240,
            input_shape=(window_length, number_of_features),
            return_sequences=False)))

# adiciona uma camada densa com 64 unidades e função de ativação linear, recebendo como entrada a saída da camada anterior
modelo.add(Dense(64))

# adiciona uma camada de dropout com 20% de probabilidade de desligar aleatoriamente as unidades da camada anterior, para evitar o sobreajuste do modelo
modelo.add(Dense(number_of_features))

# compila o modelo usando o otimizador rmsprop, a função de perda MSE e a métrica de acurácia
modelo.compile(loss='mse', optimizer='rmsprop', metrics=['accuracy'])

# treina o modelo e guarda o histórico em uma variável

history = modelo.fit(train, label,
           batch_size=100, epochs=500)


Epoch 1/500
4/4 [==============================] - 20s 406ms/step - loss: 1.0022 - accuracy: 0.0427
Epoch 2/500
4/4 [==============================] - 2s 422ms/step - loss: 0.9959 - accuracy: 0.0804
Epoch 3/500
4/4 [==============================] - 2s 404ms/step - loss: 0.9920 - accuracy: 0.1055
Epoch 4/500
4/4 [==============================] - 2s 409ms/step - loss: 0.9830 - accuracy: 0.1156
Epoch 5/500
4/4 [==============================] - 2s 399ms/step - loss: 0.9790 - accuracy: 0.1080
Epoch 6/500
4/4 [==============================] - 2s 408ms/step - loss: 0.9828 - accuracy: 0.1432
Epoch 7/500
4/4 [==============================] - 2s 417ms/step - loss: 0.9651 - accuracy: 0.1156
Epoch 8/500
4/4 [==============================] - 2s 399ms/step - loss: 0.9671 - accuracy: 0.1156
Epoch 9/500
4/4 [==============================] - 2s 411ms/step - loss: 0.9796 - accuracy: 0.1307
Epoch 10/500
4/4 [==============================] - 2s 397ms/step - loss: 0.9600 - accuracy: 0.1156
Epoch 11

In [24]:
# imprime o histórico de acurácia do modelo
print(history.history['accuracy'])

# imprime o histórico de perda do modelo
print(history.history['loss'])

# imprime o valor da acurácia final do modelo
print(history.history['accuracy'][-1])

# imprime o valor da perda final do modelo
print(history.history['loss'][-1])

[0.04271356761455536, 0.08040200918912888, 0.10552763938903809, 0.11557789146900177, 0.10804019868373871, 0.143216073513031, 0.11557789146900177, 0.11557789146900177, 0.1306532621383667, 0.11557789146900177, 0.1306532621383667, 0.11557789146900177, 0.1557788997888565, 0.13316583633422852, 0.10301507264375687, 0.12814070284366608, 0.13819095492362976, 0.12311557680368423, 0.13567839562892914, 0.13316583633422852, 0.13316583633422852, 0.143216073513031, 0.13819095492362976, 0.15829145908355713, 0.15075376629829407, 0.14824120700359344, 0.17587940394878387, 0.15829145908355713, 0.16080401837825775, 0.1658291518688202, 0.15829145908355713, 0.12311557680368423, 0.15075376629829407, 0.15829145908355713, 0.1532663255929947, 0.17336682975292206, 0.14572864770889282, 0.1658291518688202, 0.18341708183288574, 0.13567839562892914, 0.143216073513031, 0.15829145908355713, 0.14572864770889282, 0.1557788997888565, 0.18844221532344818, 0.17336682975292206, 0.17587940394878387, 0.18592964112758636, 0.16

In [25]:
# imprime o resumo do modelo
modelo.summary()



Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_12 (Bidirect  (None, 10, 480)           491520    
 ional)                                                          
                                                                 
 dropout_6 (Dropout)         (None, 10, 480)           0         
                                                                 
 bidirectional_13 (Bidirect  (None, 10, 480)           1384320   
 ional)                                                          
                                                                 
 dropout_7 (Dropout)         (None, 10, 480)           0         
                                                                 
 bidirectional_14 (Bidirect  (None, 10, 480)           1384320   
 ional)                                                          
                                                      

### Vendo o resultado do modelo

In [26]:
# salva o modelo em um arquivo HDF5
modelo.save('modelo.h5')

# carrega o modelo a partir do arquivo HDF5
modelo = tf.keras.models.load_model('modelo.h5')



c:\Users\allan\PROJETOESTATISTICA\venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Previsão

In [49]:
# cria um array vazio para armazenar os dados de teste do modelo
test = np.empty([1, window_length, number_of_features], dtype=float)

# obtém os dados de teste do dataframe transformado


# percorre as linhas do dataframe transformado e armazena os dados de teste no array criado anteriormente

test[0] = transformed_df.iloc[number_of_rows-window_length: number_of_rows, 0: number_of_features]

# imprime os dados de teste

print(test)

# imprime a forma dos dados de teste

print(test.shape)

# imprime a forma dos dados de treino

print(train.shape)

# imprime a forma dos dados de rótulo

print(label.shape)

# obtém a previsão do modelo para os dados de teste

prediction = modelo.predict(test)

# imprime a previsão do modelo

print(prediction)

# imprime a forma dos dados de teste

print(test.shape)

# imprime a forma dos dados de treino

print(train.shape)

# imprime a forma dos dados de rótulo

print(label.shape)

[[[-0.66941426 -0.93402892 -1.22685836 -1.44073145 -1.76426222
   -2.08956356 -2.27828991 -2.54269115 -2.83291739 -2.71349927
    0.61608977  0.28180073  0.58090186  0.20573556  0.63589127]
  [ 1.50684884  0.65382025  0.05170667 -0.29711582  0.51364596
    0.71116552  0.35199386  0.05215124 -0.26097637 -0.6147169
   -0.49614309 -0.94787518 -1.45059493 -1.34871089  0.63589127]
  [ 0.41871729 -0.14010434 -0.58757584 -0.86892364 -0.62530813
   -0.96927193 -0.1740629  -0.46681723 -0.26097637 -0.0900213
   -0.49614309 -0.33303723 -0.77342934 -0.57148767 -1.72268726]
  [-0.66941426 -0.93402892  0.05170667 -0.29711582 -0.62530813
   -0.96927193 -1.2261764  -0.98578571 -1.28975278  0.43467429
    0.05997334 -0.33303723 -0.77342934 -0.57148767  0.63589127]
  [ 0.41871729  2.24166942  1.33027171  0.8464998   0.51364596
    0.71116552  0.35199386  0.57111972  0.25341184  0.95936988
    0.61608977  0.28180073  0.58090186  0.20573556  0.63589127]
  [ 0.41871729 -0.14010434 -0.58757584  0.8464998   

In [51]:
# imprime a previsão do modelo desnormalizada

print(scaler.inverse_transform(prediction))

# imprime os dados de teste desnormalizados

print(scaler.inverse_transform(test[0]))


[[ 0.9930392  2.920486   3.9999146  5.443175   7.6668625  9.372391
  11.662064  13.663521  16.212952  17.624825  18.405302  18.983618
  20.7462    22.617739  25.215666 ]]
[[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 11. 19. 20. 22. 23. 25.]
 [ 3.  4.  5.  6.  9. 11. 12. 13. 14. 15. 17. 18. 19. 21. 25.]
 [ 2.  3.  4.  5.  7.  8. 11. 12. 14. 16. 17. 19. 20. 22. 23.]
 [ 1.  2.  5.  6.  7.  8.  9. 11. 12. 17. 18. 19. 20. 22. 25.]
 [ 2.  6.  7.  8.  9. 11. 12. 14. 15. 18. 19. 20. 22. 23. 25.]
 [ 2.  3.  4.  8.  9. 10. 11. 13. 17. 18. 20. 22. 23. 24. 25.]
 [ 3.  5.  9. 10. 12. 13. 14. 15. 16. 17. 18. 19. 21. 24. 25.]
 [ 1.  2.  4.  5.  6.  8.  9. 10. 12. 13. 16. 18. 23. 24. 25.]
 [ 2.  3.  6.  7.  9. 10. 11. 12. 14. 15. 16. 18. 20. 21. 25.]
 [ 4.  7.  8.  9. 10. 11. 12. 13. 14. 15. 17. 20. 22. 23. 24.]]


In [53]:
# imprime os dados de rótulo desnormalizados
print(scaler.inverse_transform(label[-1].reshape(1, -1)))


[[ 4.  7.  8.  9. 10. 11. 12. 13. 14. 15. 17. 20. 22. 23. 24.]]


### OBS.: É importante ressaltar que o código foi construído como mera forma de testar modelos, não há nenhum interesse ou benefício pessoal envolvido na sua execução. O objetivo é apenas praticar e aprender sobre as técnicas e ferramentas de machine learning, usando um problema real e desafiador como base. Não há nenhuma garantia ou pretensão de que o código seja capaz de prever os resultados da loteria com precisão ou consistência. Além disso, não se recomenda ou apoia o uso do código para fins lucrativos ou ilegais, nem se incentiva a participação em jogos de azar, que podem causar dependência e prejuízos financeiros e emocionais. O código é apenas um exercício educacional e deve ser tratado como tal.

In [59]:
# importar a biblioteca statsmodels
import statsmodels.tsa.stattools as st

# aplicar o teste de Dickey-Fuller aumentado na primeira bola sorteada
result = st.adfuller(df[0].values)
print('Estatística do teste:', result[0])
print('Valor-p:', result[1])
print('Valores críticos:', result[4])


Estatística do teste: -18.447277248565378
Valor-p: 2.1565071048804677e-30
Valores críticos: {'1%': -3.4465195891135845, '5%': -2.8686676281678634, '10%': -2.5705668101226085}


### Teste qui-quadrado

#### Hipóteses do teste
#### H0: A frequência dos números sorteados na loteria é igualmente distribuída
#### H1: A frequência dos números sorteados na loteria não é igualmente distribuída

##### O teste de qui-quadrado é um método estatístico que permite verificar se há uma associação entre duas variáveis categóricas. 

###### Neste caso, queremos saber se a frequência dos números sorteados na loteria depende ou não do número da bola. 

##### A hipótese nula do teste é que não há associação entre as variáveis, ou seja, que a frequência dos números é igualmente distribuída entre as bolas. 

##### A hipótese alternativa é que há uma associação entre as variáveis, ou seja, que a frequência dos números varia entre as bolas.

In [83]:
# Definimos uma função para interpretar os resultados do teste de qui-quadrado

def interpretar_chi2(tabela):
    # Aplicar o teste de qui-quadrado usando a função chi2_contingency
    chi2, p, dof, expected = stats.chi2_contingency(tabela)
    #Definir o nível de significância (geralmente 0,05)
    alpha = 0.05
    # Criar um texto com a interpretação dos resultados
    texto = f"O valor do qui-quadrado é {chi2:.2f} e o valor-p é {p:.4f}). " 
    if p < alpha: 
        texto += f"Como o valor-p é menor que {alpha}, podemos rejeitar a hipótese nula e concluir que há uma associação entre as variáveis." 
    else: 
        texto += f"Como o valor-p é maior que {alpha}, não podemos rejeitar a hipótese nula e não há evidências suficientes para afirmar que há uma associação entre as variáveis."
    # Retornar o texto
    return texto

# Aplicar a função para todas as bolas sorteadas

for i in range(0, 15): 
    tabela = pd.crosstab(index=df[i], columns='count')
    print(interpretar_chi2(tabela))

O valor do qui-quadrado é 0.00 e o valor-p é 1.0000). Como o valor-p é maior que 0.05, não podemos rejeitar a hipótese nula e não há evidências suficientes para afirmar que há uma associação entre as variáveis.
O valor do qui-quadrado é 0.00 e o valor-p é 1.0000). Como o valor-p é maior que 0.05, não podemos rejeitar a hipótese nula e não há evidências suficientes para afirmar que há uma associação entre as variáveis.
O valor do qui-quadrado é 0.00 e o valor-p é 1.0000). Como o valor-p é maior que 0.05, não podemos rejeitar a hipótese nula e não há evidências suficientes para afirmar que há uma associação entre as variáveis.
O valor do qui-quadrado é 0.00 e o valor-p é 1.0000). Como o valor-p é maior que 0.05, não podemos rejeitar a hipótese nula e não há evidências suficientes para afirmar que há uma associação entre as variáveis.
O valor do qui-quadrado é 0.00 e o valor-p é 1.0000). Como o valor-p é maior que 0.05, não podemos rejeitar a hipótese nula e não há evidências suficientes 

#### Obtivemos um valor do qui-quadrado de 0,00 e um valor-p de 1,00. 

###### Isso significa que os dados observados são exatamente iguais aos dados esperados sob a hipótese nula. Como o valor-p é maior que o nível de significância de 0,05, você não pode rejeitar a hipótese nula e concluir que não há evidências suficientes para afirmar que há uma associação entre as variáveis. Em outras palavras, você não encontrou nenhuma diferença estatística na distribuição dos números sorteados entre as bolas.